In [51]:
def process_line(line):
    return [
        (int(pos[0]), int(pos[1]))
        for pos in
        [pos.split(',') for pos in line.split(' -> ')]
    ]
    
data = [
    process_line(line)
    for line in [
        "498,4 -> 498,6 -> 496,6",
        "503,4 -> 502,4 -> 502,9 -> 494,9",
    ]
]

In [52]:
from pathlib import Path

input_file = Path('input')
data = [
    process_line(line)
    for line in input_file.read_text().strip().split('\n')
]

In [15]:
from itertools import tee

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [16]:
def place_rocks():
    rocks = set()
    for line in data:
        for ((x1, y1), (x2, y2)) in pairwise(line):
            if x1 == x2:
                rocks |= {(x1, yi) for yi in range(min([y1, y2]), max([y1, y2]) + 1)}
            elif y1 == y2:
                rocks |= {(xi, y1) for xi in range(min([x1, x2]), max([x1, x2]) + 1)}
    return rocks

In [17]:
def draw_map():
    for y in range(15):
        for x in range(485, 525):
            if (x, y) in rocks:
                char = '#'
            elif (x, y) in sand:
                char = 'o'
            else:
                char = '.'
            print(char, end='')
        print()

In [18]:
def sand_fall_once(pos):
    x, y = pos
    sand.discard(pos)
    filled = rocks | sand
    down = (x, y + 1)
    left = (x - 1, y + 1)
    right = (x + 1, y + 1)
    for nxt in (down, left, right):
        if nxt not in filled:
            sand.add(nxt)
            return nxt
    sand.add(pos)
    return pos

In [19]:
class TheAbyss(Exception):
    pass

def sand_fall_until_stop():
    pos = (500, 0)
    while True:
        if (new_pos := sand_fall_once(pos)) == pos:
            return
        x, y = new_pos
        if y == ABYSS:
            sand.discard(new_pos)
            raise TheAbyss()
        pos = new_pos

In [33]:
rocks = place_rocks()
ABYSS = max(y for (x, y) in rocks) + 1
sand = set()
while True:
    try:
        sand_fall_until_stop()
    except TheAbyss:
        break
        
print("Part 1:")
print(len(sand))

Part 1:
24


In [47]:
def sand_fall_once_2(pos):
    x, y = pos
    sand.discard(pos)
    filled = rocks | sand
    down = (x, y + 1)
    left = (x - 1, y + 1)
    right = (x + 1, y + 1)
    for nxt in (down, left, right):
        nx, ny = nxt
        if ny <= FLOOR and nxt not in filled:
            sand.add(nxt)
            return nxt
    sand.add(pos)
    return pos

In [48]:
class SandOverflow(Exception):
    pass

ENTRY = (500, 0)

def sand_fall_until_stop_2():
    pos = ENTRY
    while True:
        if (new_pos := sand_fall_once_2(pos)) == pos:
            if new_pos == ENTRY:
                raise SandOverflow()
            return
        pos = new_pos

In [53]:
rocks = place_rocks()
FLOOR = max(y for (x, y) in rocks) + 1
sand = set()
while True:
    sand_before = len(sand)
    try:
        sand_fall_until_stop_2()
    except SandOverflow:
        break
    sand_after = len(sand)
    if sand_before == sand_after:
        break
        
print("Part 2:")
print(len(sand))

Part 2:
25585
